In [14]:
import pandas as pd
import twint
import numpy as np
import nest_asyncio
nest_asyncio.apply()
import datetime
import ast
from newspaper import Article
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
stop_words = set(stopwords.words('english'))
import re
import swifter
from urllib.parse import urlparse

In [15]:
df = pd.read_csv("NepaliTimes_analysis.csv", parse_dates=['created_at'])

In [16]:
df

,id,created_at,urls,article,level1,level2,level3,level1_count,level_2_3_valid,level2_count,level3_count
0,1376906056824016911,2021-03-30 20:21:00+05:45,https://www.nepalitimes.com/banner/the-sun-ris...,"far-west neglected rest nepal centuries , brim...",[],[],[],0,0,0,0
1,1376875102919258113,2021-03-30 18:18:00+05:45,https://www.nepalitimes.com/opinion/educating-...,# metoo movement started 2006 activist sexual ...,[],[],[],0,0,0,0
2,1376843645316304902,2021-03-30 16:13:00+05:45,https://www.nepalitimes.com/latest/female-foot...,"monday , nepal clinched three nations cup defe...",[],[],[],0,1,0,0
3,1376810929963196416,2021-03-30 14:03:00+05:45,https://www.nepalitimes.com/editorial/devoluti...,one afternoon last month ramaroshan rural muni...,[],[],[],0,0,0,0
4,1376782744001253378,2021-03-30 12:11:00+05:45,https://www.nepalitimes.com/here-now/delayed-d...,"toiling 18 months qatar ‘ target earner ’ , ol...",[],[],[],0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
5494,1389208118660268037,2021-05-03 19:05:00+05:45,https://www.nepalitimes.com/latest/nepal-postp...,nepal ’ government decided monday postpone 12t...,"['bureau statist', 'data collect', 'nation sta...","['accur', 'proper', 'val']",[],1,1,1,0
5495,1389180779800915971,2021-05-03 17:16:22+05:45,https://www.nepalitimes.com/latest/nepal-desig...,amidst dramatic surge covid-19 infections nepa...,[],['val'],[],0,1,1,0
5496,1389144253368979461,2021-05-03 14:51:13+05:45,https://www.nepalitimes.com/opinion/how-nepal-...,try treat sick increasing hospital space ensur...,[],"['fake', 'prec']",[],0,1,1,0
5497,1389109381724205056,2021-05-03 12:32:39+05:45,https://www.nepalitimes.com/latest/kathmandu-l...,"2013 , government planners identified 83 open ...",[],[],[],0,0,0,0


In [17]:
# todo change this to func
def filter_url(url):
    return ((urlparse(url).netloc)=="www.nepalitimes.com")

In [18]:
df=df[df.urls.apply(filter_url)]

In [19]:
def clean_text(raw):
    text = BeautifulSoup(raw, 'lxml').text
    word_tokens = word_tokenize(text.lower().rstrip())
    filtered_sentence = [w for w in word_tokens if not w in stop_words]
    return ' '.join(filtered_sentence)

In [20]:
def article(url):
    try:
        article = Article(url)
        article.download()
        article.parse()
        return clean_text(article.text)
    except Exception as e:
        return 404

In [21]:
def primary_url(urls):
    if len(urls)==0:
        return 404
    return urls[0]

In [22]:
def get_new_articles(newspaper_username, since, until):
    c = twint.Config()
    c.Username = newspaper_username
    c.Pandas = True
    c.Hide_output= True
    c.Since = since
    c.Until = until
    twint.run.Search(c)
    df = twint.storage.panda.Tweets_df
    try: 
        df = df[['id', 'created_at', "urls"]]
        df['urls'] = df.urls.apply(primary_url)
        df = df[df.urls!=404]
        df = df[df.urls.apply(filter_url)]
        df['created_at'] = (pd.to_datetime(df['created_at'], unit='ms')
                         .dt.tz_localize('utc')
                         .dt.tz_convert('Asia/Kathmandu'))   
        return df
    except:
        return pd.DataFrame(columns=['id', 'created_at', "urls"])

In [23]:
data_level1 = [
    'arabl land',
    'avail data',
    'bureau statist',
    'busi survey',
    'cens publi',
    'cens pop',
    'children employ',
    'civil registr',
    'collect method',
    'commerci export',
    'complet rate',
    'consum electr',
    'consum energ',
    'data access',
    'data collect',
    'data compil',
    'data entri',
    'data manag',
    'data releas',
    'data standard',
    'data user',
    'demograph data',
    'densit popul',
    'develop data',
    'difusion dat',
    'direct statist',
    'disaggreg data',
    'electr access',
    'electr consumpt',
    'energi consumpt',
    'establish survey',
    'exchang rate',
    'extern debt',
    'fertil rate',
    'food import',
    'food product',
    'gender gap',
    'govern debt',
    'govern statist',
    'gross domest',
    'gross nation',
    'health expenditur',
    'health survey',
    'import marchandis',
    'improv data',
    'improv statist',
    'indic measur',
    'indic preci',
    'inflat rate',
    'institut statist',
    'interest payment',
    'intern tourism',
    'irrig land',
    'land use',
    'life expect',
    'livestock product',
    'merchandis export',
    'merchandis trade',
    'model statist',
    'mortal rate',
    'multilater debt',
    'nation account',
    'nation statist',
    'nation survey',
    'national brut',
    'national statist',
    'open data',
    'part revenus',
    'pay gap',
    'popul census',
    'popul growth',
    'popul rate',
    'price index',
    'produccion aliment',
    'purchas power',
    'qualiti data',
    'receit fiscal',
    'releas data',
    'revenu fiscal',
    'rural popul',
    'servic export',
    'statist agenc',
    'statist author',
    'statist avail',
    'statist committe',
    'statist data',
    'statist depart',
    'statist national',
    'statist offic',
    'statist servic',
    'statist studi',
    'survey catalogu',
    'tax payment',
    'tax revenu',
    'trade balanc',
    'unemploy rate',
    'use data',
    'water suppli',
    'youth unemploy',
    ]

data_level2 = [
    'accur',
    'adequ',
    'ambigu',
    'apropi',
    'bancal',
    'bias',
    'confiabl',
    'correct',
    'deceit',
    'deceiv',
    'decept',
    'defectu',
    'delud',
    'engan',
    'equivoc',
    'erreur',
    'erro',
    'erron',
    'errone',
    'error',
    'exact',
    'exat',
    'fake',
    'fallaci',
    'faux',
    'fiabl',
    'generaliz',
    'illus',
    'imparcial',
    'impartial',
    'imprecis',
    'improp',
    'inaccur',
    'incorrect',
    'inexact',
    'invalid',
    'limit',
    'manipul',
    'mislead',
    'mistaken',
    'parcial',
    'prec',
    'precis',
    'proper',
    'reliabl',
    'rigor',
    'rigour',
    'scientif',
    'spurious',
    'tromp',
    'trompeur',
    'unbias',
    'unreli',
    'unscientif',
    'unsound',
    'vag',
    'vagu',
    'val',
    'valid',
    ]

data_level3 = [
    'data manipul',
    'lead question',
    'manipul dat',
    'report bias',
    'sampl select',
    'sampl size',
    ]

data_level_indicator = [
    ' cpi ',
    ' fdi ',
    ' gdp ',
    ' gnp ',
    ' hdi ',
    ' wdi ',
    ]

filter_list = [' data ', ' record ', ' research ', ' statistics ',
               ' study ']

In [24]:
def level1_count(article):
    if article:
        article = str(article)
        keyword_list = []
        for word in data_level1:
            search_ = r"\b" + word.split()[0] + r"[a-zA-Z]*\s\b" \
                + word.split()[1] + r"[a-zA-Z]*"
            if re.search(search_, article):
                keyword_list.append(word)
        for word in data_level_indicator:
            if word in article:
                keyword_list.append(word)
        return keyword_list
    return []


def level2_count(article, valid):
    if valid == 1:
        article = str(article)
        keyword_list = []
        for word in data_level2:
            search_ = r"\b" + word + r"[a-zA-Z]*"
            if re.search(search_,  article):
                keyword_list.append(word)
        return keyword_list
    return []


def level3_count(article, valid):
    if valid == 1:
        article = str(article)
        keyword_list = []
        for word in data_level3:
            search_ = r"\b" + word + r"[a-zA-Z]*"
            if re.search(search_,   article):
                keyword_list.append(word)
        return keyword_list
    return []


def level_2_3_filter(article):
    article = str(article)
    for word in filter_list:
        if word in article:
            return 1
    return 0


def level_len(count_list):
    if type(count_list) == str:
        return (1 if len(count_list) > 2 else 0)
    return (1 if len(count_list) > 0 else 0)

In [12]:
df_ = get_new_articles(newspaper_username='NepaliTimes', 
                       since=df.created_at.max().strftime("%Y-%m-%d %H:%M:%S"),
                       until=datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"))

[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.


<ipython-input-9-ef6436a06e09>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['urls'] = df.urls.apply(primary_url)


In [25]:
df_

,id,created_at,urls
0,1393597720481329152,2021-05-15 21:47:43+05:45,https://www.nepalitimes.com/here-now/nepals-di...
1,1393563511330992132,2021-05-15 19:31:46+05:45,https://www.nepalitimes.com/latest/patans-red-...
2,1393514997272260611,2021-05-15 16:19:00+05:45,https://www.nepalitimes.com/latest/im-down-but...
3,1393486308496928771,2021-05-15 14:25:00+05:45,https://www.nepalitimes.com/opinion/dear-sarka...
4,1393453340747608066,2021-05-15 12:14:00+05:45,https://www.nepalitimes.com/latest/more-remitt...
5,1393425910070747137,2021-05-15 10:25:00+05:45,https://www.nepalitimes.com/banner/natures-fra...
6,1393393718153990145,2021-05-15 08:17:05+05:45,https://www.nepalitimes.com/banner/two-nepals-...
8,1393160685412962308,2021-05-14 16:51:05+05:45,https://www.nepalitimes.com/20-years-ago-this-...
9,1393141220562534401,2021-05-14 15:33:44+05:45,https://www.nepalitimes.com/banner/adding-fuel...
10,1393106019740393473,2021-05-14 13:13:52+05:45,https://www.nepalitimes.com/banner/peak-season...


In [26]:
df_['article'] = df_.urls.swifter.progress_bar(True).apply(article)

In [27]:
df_ = df_[df_['article']!=404]

In [28]:
df = df.append(df_)

In [29]:
df['level1'] = df.apply(lambda x: (level1_count(x['article']) if pd.isnull(x.level1) else x.level1), axis=1)

In [30]:
df['level1_count'] = df.level1.apply(level_len)

In [31]:
df['level_2_3_valid'] = df['article'].apply(level_2_3_filter)

In [32]:
df['level2'] = df.apply(lambda x: (level2_count(x['article'
                                    ], x['level_2_3_valid'
                                    ]) if pd.isnull(x.level2) else x.level2),
                                    axis=1)


In [33]:
df['level2_count'] = df.level2.apply(level_len)

In [34]:
df['level3'] = df.apply(lambda x: (level3_count(x['article'
                                    ], x['level_2_3_valid'
                                    ]) if pd.isnull(x.level3) else x.level3),
                                    axis=1)

In [35]:
df['level3_count'] = df.level3.apply(level_len)

In [36]:
df.level1_count.sum()/df.shape[0]

0.07102736002899077

In [37]:
df.level2_count.sum()/df.level_2_3_valid.sum()

0.6480392156862745

In [38]:
df.level3_count.sum()/df.level_2_3_valid.sum()

0.0

In [40]:
df.to_csv('NepaliTimes_analysis.csv', index=False)

In [41]:
df.created_at.max()

Timestamp('2021-05-15 21:47:43+0545', tz='Asia/Kathmandu')

In [43]:
df

,id,created_at,urls,article,level1,level2,level3,level1_count,level_2_3_valid,level2_count,level3_count
0,1376906056824016911,2021-03-30 20:21:00+05:45,https://www.nepalitimes.com/banner/the-sun-ris...,"far-west neglected rest nepal centuries , brim...",[],[],[],0,0,0,0
1,1376875102919258113,2021-03-30 18:18:00+05:45,https://www.nepalitimes.com/opinion/educating-...,# metoo movement started 2006 activist sexual ...,[],[],[],0,0,0,0
2,1376843645316304902,2021-03-30 16:13:00+05:45,https://www.nepalitimes.com/latest/female-foot...,"monday , nepal clinched three nations cup defe...",[],[],[],0,1,0,0
3,1376810929963196416,2021-03-30 14:03:00+05:45,https://www.nepalitimes.com/editorial/devoluti...,one afternoon last month ramaroshan rural muni...,[],[],[],0,0,0,0
4,1376782744001253378,2021-03-30 12:11:00+05:45,https://www.nepalitimes.com/here-now/delayed-d...,"toiling 18 months qatar ‘ target earner ’ , ol...",[],[],[],0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
16,1392846877482049543,2021-05-13 20:04:08+05:45,https://www.nepalitimes.com/latest/internation...,nepal ’ covid-19 crisis figured us senate fore...,[],[],[],0,0,0,0
19,1392773863579131908,2021-05-13 15:14:00+05:45,https://www.nepalitimes.com/latest/more-remitt...,remittances nepali workers abroad continued de...,[],[],[],0,0,0,0
20,1392742500427726853,2021-05-13 13:09:22+05:45,https://www.nepalitimes.com/banner/adding-fuel...,"even normal times , depot operates 24 hours en...",[],[],[],0,0,0,0
21,1392694522975461379,2021-05-13 09:58:43+05:45,https://www.nepalitimes.com/here-now/at-the-co...,"nepal ’ rural health system , already underfun...",[],[],[],0,0,0,0


In [44]:
df[['urls',
 'created_at',
 'level1_count',
 'level_2_3_valid',
 'level2_count',
 'level3_count']].to_csv("NepaliTimes_flask.csv", index=False)

In [45]:
df

,id,created_at,urls,article,level1,level2,level3,level1_count,level_2_3_valid,level2_count,level3_count
0,1376906056824016911,2021-03-30 20:21:00+05:45,https://www.nepalitimes.com/banner/the-sun-ris...,"far-west neglected rest nepal centuries , brim...",[],[],[],0,0,0,0
1,1376875102919258113,2021-03-30 18:18:00+05:45,https://www.nepalitimes.com/opinion/educating-...,# metoo movement started 2006 activist sexual ...,[],[],[],0,0,0,0
2,1376843645316304902,2021-03-30 16:13:00+05:45,https://www.nepalitimes.com/latest/female-foot...,"monday , nepal clinched three nations cup defe...",[],[],[],0,1,0,0
3,1376810929963196416,2021-03-30 14:03:00+05:45,https://www.nepalitimes.com/editorial/devoluti...,one afternoon last month ramaroshan rural muni...,[],[],[],0,0,0,0
4,1376782744001253378,2021-03-30 12:11:00+05:45,https://www.nepalitimes.com/here-now/delayed-d...,"toiling 18 months qatar ‘ target earner ’ , ol...",[],[],[],0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
16,1392846877482049543,2021-05-13 20:04:08+05:45,https://www.nepalitimes.com/latest/internation...,nepal ’ covid-19 crisis figured us senate fore...,[],[],[],0,0,0,0
19,1392773863579131908,2021-05-13 15:14:00+05:45,https://www.nepalitimes.com/latest/more-remitt...,remittances nepali workers abroad continued de...,[],[],[],0,0,0,0
20,1392742500427726853,2021-05-13 13:09:22+05:45,https://www.nepalitimes.com/banner/adding-fuel...,"even normal times , depot operates 24 hours en...",[],[],[],0,0,0,0
21,1392694522975461379,2021-05-13 09:58:43+05:45,https://www.nepalitimes.com/here-now/at-the-co...,"nepal ’ rural health system , already underfun...",[],[],[],0,0,0,0
